## 📌 **Django REST Framework (DRF) - Creando APIs para frontend y apps móviles**  

El objetivo de este tema es que aprendas a construir **APIs REST** en Django usando **Django REST Framework (DRF)**. Veremos cómo convertir nuestros modelos en **endpoints JSON**, para que puedan ser consumidos por aplicaciones web, móviles o cualquier otro cliente.  

---

## 🔹 **1. ¿Qué es una API y por qué usar Django REST Framework?**  

Una **API (Application Programming Interface)** permite que diferentes aplicaciones se comuniquen entre sí.  

💡 **Ejemplo de uso:**  
Un backend en Django que maneja usuarios y publicaciones, y un frontend en **React** o una app móvil en **Flutter** que consume los datos.  

📌 **Ventajas de usar DRF en Django:**  
✔️ Convierte modelos Django en **endpoints JSON** de manera sencilla.  
✔️ Facilita la autenticación con **tokens o JWT**.  
✔️ Permite definir permisos y protecciones de seguridad.  

---

## 🔹 **2. Instalando Django REST Framework**  

Ejecutamos el siguiente comando para instalar DRF:  

```bash
pip install djangorestframework
```

Luego, agregamos `'rest_framework'` en `INSTALLED_APPS` dentro de `settings.py`:  

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'rest_framework',  # Agregar esta línea
]
```

---

## 🔹 **3. Creando una API REST en Django paso a paso**  

### 📍 **Paso 1: Definir el modelo (`models.py`)**  

Vamos a usar el modelo `Publicacion` que creamos antes:  

```python
from django.db import models

class Publicacion(models.Model):
    titulo = models.CharField(max_length=200)
    contenido = models.TextField()
    fecha_creacion = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return self.titulo
```

📍 **Ejecutamos las migraciones:**  

```bash
python manage.py makemigrations
python manage.py migrate
```

---

### 📍 **Paso 2: Crear un `serializer` para convertir el modelo a JSON**  

Creamos un archivo `serializers.py` dentro de la mi_app y agregamos:  

```python
from rest_framework import serializers
from .models import Publicacion

class PublicacionSerializer(serializers.ModelSerializer):
    class Meta:
        model = Publicacion
        fields = '__all__'  # Incluir todos los campos
```

📌 **¿Qué hace este `serializer`?**  
✔️ Convierte objetos Django en **JSON** para la API.  
✔️ También permite convertir datos JSON en objetos Django.  

---

### 📍 **Paso 3: Crear la vista de la API (`views.py`)**  

Ahora creamos una vista para devolver los datos en formato JSON usando `APIView`:  

```python
from rest_framework.response import Response
from rest_framework.views import APIView
from .models import Publicacion
from .serializers import PublicacionSerializer

class PublicacionList(APIView):
    def get(self, request):
        publicaciones = Publicacion.objects.all()
        serializer = PublicacionSerializer(publicaciones, many=True)
        return Response(serializer.data)
```

📌 **Explicación:**  
✔️ `get()` obtiene todas las publicaciones de la base de datos.  
✔️ `PublicacionSerializer(publicaciones, many=True)` convierte los datos a JSON.  
✔️ `Response(serializer.data)` devuelve la respuesta en JSON.  

---

### 📍 **Paso 4: Definir la URL de la API (`urls.py`)**  

En `urls.py` de la app, registramos la nueva ruta de la API:  

```python
from django.urls import path
from .views import PublicacionList

urlpatterns = [
    path('api/publicaciones/', PublicacionList.as_view(), name='api_publicaciones'),
]
```

Ahora, si visitamos `http://127.0.0.1:8000/api/publicaciones/`, veremos un JSON con todas las publicaciones.  

---

## 🔹 **4. CRUD Completo con `ViewSets` y `Routers`**  

Usar `APIView` funciona bien, pero DRF nos ofrece una manera más eficiente con **ViewSets** y **Routers**.  

### 📍 **Paso 1: Usar `ViewSet` en `views.py`**  

```python
from rest_framework import viewsets
from .models import Publicacion
from .serializers import PublicacionSerializer

class PublicacionViewSet(viewsets.ModelViewSet):
    queryset = Publicacion.objects.all()
    serializer_class = PublicacionSerializer
```

📌 **¿Qué hace `ModelViewSet`?**  
✔️ Automáticamente genera las funciones `GET`, `POST`, `PUT` y `DELETE`.  
✔️ Evita que tengamos que escribirlas manualmente.  

---

### 📍 **Paso 2: Usar `routers` en `urls.py`**  

```python
from django.urls import path, include
from rest_framework.routers import DefaultRouter
from .views import PublicacionViewSet

router = DefaultRouter()
router.register(r'publicaciones', PublicacionViewSet)

urlpatterns = [
    path('api/', include(router.urls)),
]
```


1. Ve a tu `admin.py` y registra el modelo:

   ```python
   from django.contrib import admin
   from .models import Publicacion

   admin.site.register(Publicacion)
   ```

3. Ve a `http://127.0.0.1:8000/admin/`, inicia sesión, y crea una publicación desde ahí.

4. Vuelve a `http://127.0.0.1:8000/api/publicaciones/` y verás que aparece en la lista.


Ahora, `http://127.0.0.1:8000/api/publicaciones/` soporta **todas** las operaciones de un CRUD:  

✔️ `GET /api/publicaciones/` → Listar publicaciones.  
✔️ `POST /api/publicaciones/` → Crear una nueva publicación.  
✔️ `GET /api/publicaciones/1/` → Obtener detalle de una publicación.  
✔️ `PUT /api/publicaciones/1/` → Editar una publicación.  
✔️ `DELETE /api/publicaciones/1/` → Eliminar una publicación.  

---

## 🔹 **5. Agregando autenticación con DRF**  

Para proteger las APIs, podemos exigir autenticación con `TokenAuthentication`:  

📍 **Paso 1: Agregar `rest_framework.authtoken` en `INSTALLED_APPS`**  

```python
INSTALLED_APPS = [
    'rest_framework',
    'rest_framework.authtoken',
]
```

📍 **Paso 2: Migrar la base de datos para crear tokens**  

```bash
python manage.py migrate
```

📍 **Paso 3: Configurar `DEFAULT_AUTHENTICATION_CLASSES` en `settings.py`**  

```python
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework.authentication.TokenAuthentication',
    ],
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.IsAuthenticated',
    ]
}
```

📍 **Paso 4: Generar un token para un usuario**  

```bash
python manage.py shell
```

### Cambiar `admin` por su nombre de administrador

```python
from rest_framework.authtoken.models import Token
from django.contrib.auth.models import User

usuario = User.objects.get(username='admin')
token, created = Token.objects.get_or_create(user=usuario)
print(token.key)
```

Ahora, en las solicitudes a la API, debemos enviar el token en los headers:  

```
Authorization: Token <TU_TOKEN>
```

Si no enviamos el token, la API rechazará la solicitud.  

---

## ✅ **Resumen del flujo de trabajo con DRF**  

1️⃣ Instalamos DRF (`pip install djangorestframework`).  
2️⃣ Creamos un `serializer.py` para convertir modelos en JSON.  
3️⃣ Creamos vistas con `APIView` o `ViewSet`.  
4️⃣ Definimos rutas usando `path()` o `routers`.  
5️⃣ Agregamos autenticación con `TokenAuthentication`.  